In [1]:
from learning import A2CLearner
from Critic_and_Actor import Critic, Actor, Actor_best, Critic_best
from tensorboardX import SummaryWriter
import torch
import gym
from datetime import datetime
env = gym.make('BipedalWalker-v3')

state_dim = env.observation_space.shape[0]
n_actions = env.action_space.shape[0]

## 여기서 새로정의 및 저장하는 코드만들기 
max_episode = 10000#학습당 사용할 에피소드의 수
use_best = ['best', 'origin']
learning_rate_actor = [4e-4,4e-3,4e-5]
learning_rate_critic = [4e-3,4e-3,4e-4]
gammas = [0.9, 0.8, 0.7]
variance
for model_mode in use_best:
    for learning_index  in range(len(learning_rate_actor)):
        for gamma in gammas:
            now = datetime.now()
            dt_string = now.strftime("%d_%d %H:%M:%S ")
            writer = SummaryWriter('runs/'+dt_string+
                                   " model_"+model_mode+"-"+
                                   " lr_actor_"+str(learning_rate_actor[learning_index])+"-"+
                                   " lr_critic_"+str(learning_rate_critic[learning_index])+"-"+
                                   " gamma_"+str(gamma)
                                  )

            #사용하는 모델 
            if model_mode =="origin":
                actor = Actor(state_dim, n_actions).cuda() 
                critic = Critic(state_dim).cuda() 
            elif model_mode =="best":#모델의 레이어 수 및 상태가 많아지고, 편차도 커짐 
                actor = Actor_best(state_dim, n_actions).cuda() 
                critic = Critic_best(state_dim).cuda() 

            learner = A2CLearner(actor, critic)
            learner.A2C_env_run(env, max_episode, writer)

            #학습다돌리고 10번정도 확인하기
            
            torch.save(actor.state_dict(),
                       "./model_weight/"+dt_string+model_mode+
                       " lr_actor_"+str(learning_rate_actor[learning_index])+
                       "-"+" gamma_"+str(gamma)+
                       " actor_weight.pth")
            torch.save(critic.state_dict(),
                       "./model_weight/"+dt_string+model_mode+
                       " lr_critic_"+str(learning_rate_critic[learning_index])+
                       "-"+" gamma_"+str(gamma)+
                       " critic_weight.pth")
            for j in range(10):
              learner.view(env)
            env.close()
            writer.close()





In [27]:
from model_tester import model_tester
model_tester("13_13 06:14:15 origin lr_actor_4e-05- gamma_0.7 actor_weight.pth","origin")

./model_weight/13_13 06:14:15 origin lr_actor_4e-05- gamma_0.7 actor_weight.pth
